In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import torch
import json

In [ ]:
train=pd.read_json(r'../input/train.json')
test=pd.read_json(r'../input/test.json')

In [ ]:
train.head()

In [ ]:
train['ingredients_unrolled']=train['ingredients'].apply(lambda x :','.join(x))
train.head()

In [ ]:
# finding unique words in the ingredients
unique_words=dict()
for row in train['ingredients_unrolled']:
    for w in row.split(','):
        if w not in unique_words:
            unique_words[w]=1
        else:
            unique_words[w]+=1

In [ ]:
len(unique_words)

In [ ]:
words_count=pd.Series(unique_words)
words_count.head()

In [ ]:
from string import digits
#remove digits and percentage symbols from the text
s='abc123%'
digits+='%()oz'
print(digits)
remove_digits=str.maketrans('','',digits)
def clean_numbers_and_percentage(X):
    return X.translate(remove_digits)

In [ ]:
train['ingredients_unrolled_cleaned']=train['ingredients_unrolled'].apply(clean_numbers_and_percentage)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(tokenizer=lambda x:x.split(','))
ing_vectors=tfidf.fit_transform(train['ingredients_unrolled'])

In [ ]:
tfidf.get_feature_names()[:1000]

In [ ]:
tfidf=TfidfVectorizer(tokenizer=lambda x:x.split(','))
ing_vectors=tfidf.fit_transform(train['ingredients_unrolled_cleaned']).toarray()
#tfidf.get_feature_names()[-100:]

In [ ]:
tfidf.get_feature_names()[-1000:]

In [ ]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
cusine_count=train['cuisine'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
plt.bar(cusine_count.index,cusine_count.values)
plt.title('Cusines count')
plt.xlabel('cusines')
plt.ylabel('count')
plt.xticks(rotation='vertical')
plt.show()


In [ ]:
train['cuisine_encoded']=train['cuisine'].astype('category').cat.codes

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
ing_dim2=pca.fit_transform(ing_vectors)

In [ ]:
plt.scatter(ing_dim2[:,0],ing_dim2[:,1],c=train['cuisine_encoded'])
plt.xlabel('pca 1')
plt.ylabel('pca 2')
plt.colorbar()
plt.show()

In [ ]:
label_map=cusine_count.to_dict()
label_map

In [ ]:
y_train=train['cuisine'].map(label_map)

In [ ]:
from sklearn.model_selection import cross_val_score,cross_validate

In [ ]:
scores=cross_validate(RandomForestClassifier(n_estimators=100),X=ing_vectors,y=y_train,cv=10)

In [ ]:
scores
plt.plot(scores['test_score'])
plt.title('test score')
plt.show()
plt.plot(scores['train_score'])
plt.title('train score')
plt.show()


In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
scores=cross_validate(LogisticRegression(),X=ing_vectors,y=y_train,cv=10)

In [ ]:
plt.plot(scores['test_score'])
plt.title('test score')
plt.show()
plt.plot(scores['train_score'])
plt.title('train score')
plt.show()

In [ ]:
import xgboost

In [ ]:
gclf=xgboost.XGBClassifier()

In [ ]:
cros

In [ ]:
scores=cross_validate(gclf,X=ing_vectors,y=y_train,cv=10)
plt.plot(scores['test_score'])
plt.title('test score')
plt.show()
plt.plot(scores['train_score'])
plt.title('train score')
plt.show()

In [ ]:
import lightgbm as lgb
param = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
    }

In [ ]:
#lgbm=lightgbm.LGBMClassifier()
def fit_lgbm(X,y,param=param,num_round=10):
    
    evals_result = {}
    train_data = lgb.Dataset(X, label=y)
    test_data = train_data.create_valid(train_data)
    lgb_model = lgb.train(param, 
                      train_data, 
                      num_round, 
                      valid_sets = [test_data], 
                     
                      evals_result = evals_result)

    return lgb_model

In [ ]:
ing_vectors

In [ ]:
#model=fit_lgbm(ing_vectors,y_train)

In [ ]:
#scores=cross_validate(lgbm,X=ing_vectors,y=y_train,cv=10)
plt.plot(scores['test_score'])
plt.title('test score')
plt.show()
plt.plot(scores['train_score'])
plt.title('test score')
plt.show()

In [ ]:
test.head()

In [ ]:
sub=pd.read_csv(r'../input/sample_submission.csv')
sub.head()

In [ ]:
def clean_test_text():
    test['ingredients_unrolled']=test['ingredients'].apply(lambda x :','.join(x))
    test['ingredients_unrolled_cleaned']=test['ingredients_unrolled'].apply(clean_numbers_and_percentage)
    return tfidf.transform(test['ingredients_unrolled_cleaned']).toarray()
    

In [ ]:
ing_vectors.shape

In [ ]:
test_ing_vec=clean_test_text()
test_ing_vec.shape

In [ ]:
lr=LogisticRegression()
rf=RandomForestClassifier(n_est)
gclf=xgboost.XGBClassifier()

In [ ]:
y_train=y_train.astype('category')

In [ ]:
from sklearn.ensemble import VotingClassifier
eclf1 = VotingClassifier(estimators=[('lr', lr), ('rf', rf), ('gnb', gclf)], voting='hard',n_jobs=-1)

In [ ]:
#eclf1 = eclf1.fit(ing_vectors, y_train)